In [ ]:
pip install --upgrade pip

In [ ]:
pip install pytrends

In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install wordcloud

In [ ]:
pip install seaborn

In [ ]:
pip install plotly

In [ ]:
from pytrends.request import TrendReq
import pandas as pd
from IPython.display import display, HTML

# Create pytrends object
pytrends = TrendReq()

# Read keyword input from the user
keyword_input = input("Enter a keyword value: ")

# Build payload to get related queries for the main keyword
try:
    pytrends.build_payload(kw_list=[keyword_input], geo='US', timeframe='today 12-m')
    related_queries_main = pytrends.related_queries().values()
except Exception as e:
    print("Error occurred while fetching related queries:", e)
    related_queries_main = []

# Extract queries from the dictionary values for the main keyword
queries = [query_data['top']['query'].tolist() for query_data in related_queries_main if query_data['top'] is not None]
queries = [query for sublist in queries for query in sublist]

# Define the number of results per batch
results_per_batch = 25

# List to store all related queries
related_queries_all = []

# Process queries in batches
for query in queries:
    try:
        pytrends.build_payload(kw_list=[query], geo='US', timeframe='today 12-m')
        related_queries = pytrends.related_queries().values()
        related_queries_all.extend([
            {'query': related_query, 'value': value}
            for query_data in related_queries
            if query_data['top'] is not None
            for related_query, value in zip(query_data['top']['query'].tolist(), query_data['top']['value'].tolist())
        ])
    except Exception as e:
        print("Error occurred while processing query:", query, e)

# Aggregate and sum related queries
related_queries_agg = {}
for item in related_queries_all:
    query = item['query']
    value = item['value']
    if query in related_queries_agg:
        related_queries_agg[query] += value
    else:
        related_queries_agg[query] = value

# Sort the related queries by value in descending order
related_queries_all_sorted = [{'query': k, 'value': v} for k, v in sorted(related_queries_agg.items(), key=lambda item: item[1], reverse=True)]

# Convert the sorted related queries data into a pandas DataFrame
df = pd.DataFrame(related_queries_all_sorted)

# Export data to CSV with user-specified filename
csv_filename = f"{keyword_input}.csv"
df.to_csv(csv_filename, index=False)
print(f"Data exported to {csv_filename}")
df

In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import plotly.express as px
from requests.exceptions import HTTPError
import time

# Create pytrends object
pytrends = TrendReq()

# Read keyword input from the user
keyword = input("Enter a keyword value: ")

# Build payload to get search interest by region
pytrends.build_payload(kw_list=[keyword], geo='US', timeframe='today 12-m')

# Get search interest by region with retries
state_data = None
for attempt in range(5):
    try:
        state_data = pytrends.interest_by_region(resolution='REGION', inc_low_vol=True, inc_geo_code=False)
        break  # Break the loop if successful
    except HTTPError as e:
        print(f"HTTP Error occurred: {e}")
        print("Retrying...")
        time.sleep(5)  # Wait for 5 seconds before retrying

if state_data is None:
    print("Failed to retrieve data after multiple attempts.")
    exit()

# Batch process state name retrieval
state_codes = state_data.index.tolist()
try:
    suggestions = pytrends.suggestions(','.join(state_codes))
    state_name_dict = {item['mid']: item['title'] for item in suggestions}
except Exception as e:
    print(f"Error occurred while retrieving state names: {e}")
    state_name_dict = {}

# Convert state codes to state names
state_names = [state_name_dict.get(state_code, state_code) for state_code in state_data.index]
state_data['State'] = state_names

# Sort state-wise search interest data by values in descending order
state_data_sorted = state_data.sort_values(by=keyword, ascending=False)

# Plot state-wise keyword search interest using Plotly
fig = px.bar(state_data_sorted, x=state_data_sorted.index, y=keyword, color=keyword,
             title=f'State-wise Search Interest for the Keyword "{keyword}" in the US (Past Year)',
             labels={'value': 'Search Interest', 'index': 'State'},
             template='plotly_white')
fig.update_xaxes(tickangle=-45)
fig.show()

# Export data to CSV with user-specified filename
csv_filename = f"{keyword}_search_interest_stateWise.csv"
state_data_sorted.to_csv(csv_filename)
print(f"State-wise search interest data exported to {csv_filename}")


In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import plotly.graph_objects as go
from requests.exceptions import HTTPError
import time

# Suppress FutureWarning raised by Pandas
pd.set_option('future.no_silent_downcasting', True)

# Create pytrends object
pytrends = TrendReq()

# Read keyword input from the user
keyword = input("Enter a keyword value: ")

# Build payload to get search interest over time for the keyword in the US
pytrends.build_payload(kw_list=[keyword], geo='US', timeframe='today 12-m')

# Get search interest over time
interest_over_time_df = None
for attempt in range(5):
    try:
        interest_over_time_df = pytrends.interest_over_time()
        break  # Break the loop if successful
    except HTTPError as e:
        print(f"HTTP Error occurred: {e}")
        print("Retrying...")
        time.sleep(5)  # Wait for 5 seconds before retrying

if interest_over_time_df is None:
    print("Failed to retrieve data after multiple attempts.")
    exit()

# Create an interactive plot using Plotly
fig = go.Figure()

# Add trace for keyword search interest over time
fig.add_trace(go.Scatter(x=interest_over_time_df.index, y=interest_over_time_df[keyword],
                         mode='lines',
                         name=f'Search Interest for "{keyword}"'))

# Update layout for better labeling and interactivity
fig.update_layout(title=f'Search Interest Over Time for the Keyword "{keyword}" in the US (Past Year)',
                  xaxis_title='Date',
                  yaxis_title='Search Interest',
                  xaxis=dict(tickangle=-45, tickfont=dict(size=10)),
                  yaxis=dict(tickfont=dict(size=10)),
                  template='plotly_white')

# Show the interactive plot
fig.show()

# Export data to CSV with user-specified filename
csv_filename = f"{keyword}_search_interest_over_time.csv"
interest_over_time_df.to_csv(csv_filename)
print(f"Search interest over time data exported to {csv_filename}")


In [ ]:
import plotly.express as px

# Display the top 15 related queries
top_15_queries = df.head(15)

# Create an interactive pie chart using Plotly
fig = px.pie(top_15_queries, values='value', names='query', title='Top 15 Related Queries Distribution',
             labels={'value': 'Percentage', 'query': 'Query'}, hole=0.0)  # Set hole size to 0 for full circle

# Adjust layout to make it square and place color label on the right
fig.update_layout(
    autosize=False,
    width=900,  # Set width and height to make it square
    height=600,
    legend=dict(
        orientation="v",  # Place legend vertically
        xanchor="left",   # Anchor to the left side
        yanchor="middle",  # Anchor to the middle
        y=0.5,  # Center the legend vertically
        x=1.1,  # Adjust the x-position to place it on the right
        title_text="Query",  # Add legend title
    )
)

# Show the interactive plot
fig.show()


In [ ]:
from wordcloud import WordCloud
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Generate the word cloud
wordcloud = WordCloud(
    width=1920,  # HD width
    height=1080,  # HD height
    background_color='white',
    colormap='viridis',  # color map for the word cloud
    contour_color='steelblue',  # contour color
    contour_width=2,  # contour width
    max_words=df.shape[0],  # maximum number of words to display
    prefer_horizontal=0.8,  # prefer horizontal orientation of words
    min_font_size=10,  # minimum font size
    max_font_size=200,  # maximum font size for HD clarity
    random_state=42  # random state for reproducibility
).generate(' '.join(df['query']))

# Convert word cloud to plotly format
wordcloud_plotly = go.Image(z=wordcloud.to_array(), hoverinfo='none')

# Create subplots
fig = make_subplots(rows=1, cols=1)

# Add word cloud to subplot
fig.add_trace(wordcloud_plotly, row=1, col=1)

# Update layout
fig.update_layout(title_text='Related Queries Word Cloud', title_x=0.5, width=1280, height=720)

# Show the interactive plot
fig.show()


In [ ]:
import plotly.express as px
import pandas as pd

# Sort the DataFrame by the "value" column in descending order
sorted_df = df.sort_values(by='value', ascending=False)

# Create an interactive violin plot using Plotly
fig = px.violin(sorted_df, y='value', box=True, points="all", title='Violin Plot of Search Interest (Sorted by Value)',
                labels={'value'})

# Customize layout
fig.update_layout(
    xaxis_title='Distribution',
    yaxis_title='Search Interest',
    font=dict(size=14),
    template='plotly_white'
)

# Show the interactive plot
fig.show()


In [ ]:
import plotly.express as px
import pandas as pd
import seaborn as sns

# Set a consistent color palette
palette = sns.color_palette("viridis", n_colors=len(df))

# Create a scatter plot using Plotly
fig = px.scatter(df, x=df.index, y='value', color='query', color_discrete_sequence=palette, 
                 title='Scatter Plot of Search Interest', labels={'value': 'Search Interest', 'index': 'Query Index'})

# Customize layout
fig.update_layout(
    xaxis_title='Query Index',
    yaxis_title='Search Interest',
    font=dict(size=14),
    legend=dict(
        title='Query',
        yanchor='top',
        y=0.95,
        xanchor='left',
        x=0.75
    ),
    template='plotly_white'
)

# Show the interactive plot
fig.show()
